# Multi-Agent System with Azure AI Search using Agent Framework

Welcome! This notebook will walk you through creating a **multi-agent system** using the **Microsoft Agent Framework** with **Azure AI Search** integration. You'll learn how to build specialized agents that can search through product data and collaborate to provide comprehensive customer service.

## What You'll Learn
- How to configure Azure AI Search with the Microsoft Agent Framework
- How to create multiple specialized agents with search capabilities
- How to use `HostedFileSearchTool` for Azure AI Search integration
- How to orchestrate agents to work together on customer inquiries
- How to authenticate with Azure using `AzureCliCredential`

## Scenario: Zava Hardware Store Assistant Team with Product Search
We'll create a team of two agents that can search the **zava-products** index:
1. **Product Inventory Agent**: Technical expert with search access for specs, availability, and product details
2. **Customer Service Agent**: Friendly helper with search access for personalized recommendations

Let's get started! 🚀

---

## Step 1: Verify Required Python Packages

The dev container has already installed the necessary Python packages for you:
- `agent-framework`: The Microsoft Agent Framework SDK
- `azure-identity`: For authentication with Azure

Let's verify these packages are available and check their versions.

In [ ]:
# Verify required packages are installed
import importlib.metadata

try:
    agent_framework_version = importlib.metadata.version('agent-framework')
    azure_identity_version = importlib.metadata.version('azure-identity')
    
    print("✅ All required packages are installed!")
    print(f"📦 agent-framework: {agent_framework_version}")
    print(f"📦 azure-identity: {azure_identity_version}")
except importlib.metadata.PackageNotFoundError as e:
    print(f"❌ Missing package: {e}")
    print("Please ensure the dev container has been properly set up.")

## Step 2: Import Required Libraries

Now let's import all the libraries we'll need for this tutorial:
- `os`: For reading environment variables
- `asyncio`: For running async agent operations
- `ChatAgent`: The main agent class from Microsoft Agent Framework
- `AzureAIAgentClient`: Client for connecting to Azure AI services with search capabilities
- `HostedFileSearchTool`: Tool that enables agents to search Azure AI Search indexes
- `AzureCliCredential`: For Azure CLI-based authentication

In [ ]:
import os
import asyncio
from agent_framework import ChatAgent, HostedFileSearchTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All libraries imported successfully!")

## Step 3: Set Up Environment Variables

You'll need environment variables that should already be configured in your `.env` file:

1. **AZURE_EXISTING_AIPROJECT_ENDPOINT**: Your Azure AI Foundry project endpoint
   - Format: `https://<AIFoundryResourceName>.services.ai.azure.com/api/projects/<ProjectName>`
   - Find it in the Azure AI Foundry portal under **Overview** > **Libraries**

2. **AZURE_OPENAI_DEPLOYMENT**: The name of your deployed chat model
   - Find it in the Azure AI Foundry portal under **Models + Endpoints**
   - Common examples: `gpt-4o`, `gpt-4o-mini`, `gpt-4.1`

3. **AZURE_AI_SEARCH_INDEX_NAME**: The name of your search index (should be "zava-products")

> **Note**: These environment variables should already be set from the initial lab setup. If they're not found, please run the setup notebook in `1-Begin-Here/2-validate/` first.

In [ ]:
# Get environment variables
project_endpoint = os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT")
model_deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT") or os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
search_index_name = os.environ.get("AZURE_AI_SEARCH_INDEX_NAME") or os.environ.get("AZURE_SEARCH_INDEX_NAME")

# Verify that the variables are set
if not project_endpoint:
    raise ValueError("❌ AZURE_EXISTING_AIPROJECT_ENDPOINT environment variable is not set!")
if not model_deployment_name:
    raise ValueError("❌ AZURE_OPENAI_DEPLOYMENT environment variable is not set!")
if not search_index_name:
    raise ValueError("❌ AZURE_AI_SEARCH_INDEX_NAME environment variable is not set!")

print("✅ Environment variables loaded successfully!")
print(f"📍 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment_name}")
print(f"🔍 Search Index: {search_index_name}")

## Step 4: Verify Azure Credentials

Before creating agents, let's verify that Azure CLI credentials are available. We'll use `AzureCliCredential` which uses your `az login` session.

In [ ]:
# Create credential (will be used throughout the notebook)
credential = AzureCliCredential()

# Verify it works by getting a token
token = await credential.get_token("https://cognitiveservices.azure.com/.default")

print("✅ Azure CLI credentials verified successfully!")
print(f"🔐 Token acquired (expires: {token.expires_on})")
print("\n💡 Using AzureCliCredential (from 'az login' session)")

## Step 5: Create the Azure AI Agent Client

The `AzureAIAgentClient` is the connection between your agents and Azure AI services. It handles:
- Authentication with Azure
- Routing requests to your deployed model
- Managing the conversation state
- **Connecting to Azure AI Search for product data**

Let's create a client that will be shared across all agents in our team.

> **Documentation**: [Azure AI Agent Client](https://learn.microsoft.com/en-us/agent-framework/azure-ai-agent-client)

In [ ]:
# Create Azure AI Agent Client
# Note: We need to set the environment variables for the client
os.environ["AZURE_AI_PROJECT_ENDPOINT"] = project_endpoint
os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"] = model_deployment_name

# Create the client (will be used by all agents)
agent_client = AzureAIAgentClient(async_credential=credential)

print("✅ Azure AI Agent Client created successfully!")
print(f"📡 Connected to: {project_endpoint}")
print(f"🤖 Using model: {model_deployment_name}")

## Step 6: Configure Azure AI Search Tool

Now we'll create a **search tool** that enables our agents to search through the `zava-products` index. This tool uses the `HostedFileSearchTool` configured for Azure AI Search.

The tool will:
- Automatically connect to your Azure AI Search service
- Search through the specified index (`zava-products`)
- Return relevant product information based on queries
- Support different search configurations (query type, result count, filters)

In [ ]:
# Create Azure AI Search tool for the zava-products index
# You can change query_type to "simple", "semantic", or "hybrid"
search_tool = HostedFileSearchTool(
    additional_properties={
        "index_name": search_index_name,  # The zava-products index
        "query_type": "simple",           # Options: "simple", "semantic", "hybrid"
        "top_k": 5,                       # Return top 5 most relevant results
        # Optional: Add filters for more specific searches
        # "filter": "price lt 100",       # Example: products under $100
    }
)

print("✅ Azure AI Search tool configured successfully!")
print(f"🔍 Index: {search_index_name}")
print(f"📊 Search Type: semantic")
print(f"📈 Max Results: 5")
print("\n💡 This tool will allow agents to search through the product catalog!")
print("\n📝 Available query types:")
print("   • simple   - Fast keyword search (BM25)")
print("   • semantic - AI-powered natural language search (requires Basic tier+)")
print("   • hybrid   - Combined keyword + vector search (best results)")

### Understanding the zava-products Index

The `zava-products` index was created during setup with the following structure:

**Fields:**
- `id` - Unique product identifier
- `content` - Product description (searchable text)
- `title` - Product name (searchable text)
- `price` - Product price (filterable, sortable)
- `stock` - Stock quantity (filterable, sortable)
- `contentVector` - 1536-dimensional embeddings for semantic/vector search

**Query Types:**
- **Simple**: Searches only the text fields (`content`, `title`) using keyword matching
- **Semantic**: Adds AI understanding to interpret query intent and meaning
- **Hybrid**: Searches both text fields AND vector embeddings, merging results for best relevance

## Step 7: Create Specialized Agents with Search Capabilities

Now we'll create a **multi-agent team** with two specialized agents for Zava Hardware Store. Each agent will have access to the Azure AI Search tool to query product data:

### 📦 Product Inventory Agent
- **Role**: Technical product specialist with inventory management expertise
- **Search Capability**: Can search for product specs, availability, and alternatives
- **Capabilities**: 
  - Provides detailed technical specifications from the product database
  - Checks product availability and stock levels via search
  - Offers alternative products when stock is low or unavailable
- **Personality**: Professional, detail-oriented, technical, solution-focused

### 😊 Customer Service Agent  
- **Role**: Friendly customer support specialist
- **Search Capability**: Can search for products matching customer needs
- **Capabilities**: 
  - Helps with general inquiries and personalized recommendations
  - Provides warm, engaging customer service with emoji
  - Follows a structured response format with follow-up questions
- **Personality**: Polite, approachable, customer-focused, conversational

Each agent has its own instructions and personality, but they both share the same Azure AI backend and search tool.

> **Note**: We're creating agents using the `ChatAgent` class with the `tools` parameter. The agents are not using async context managers here because we want them to persist across multiple cells for testing. In production, you might use `async with` for automatic cleanup.

### Define Product Inventory Agent Instructions

First, let's define the detailed instructions for our Product Inventory Agent. This agent will be technical, detail-oriented, and focused on providing accurate product specifications from search results.

In [ ]:
# Product Inventory Agent Instructions
product_inventory_instructions = """You are a Product Inventory Agent for Zava, a hardware and home improvement company.

Your role:
- Search the product database to provide detailed technical specifications for tools, hardware, and paint products
- Use the search tool to find specific details like: coverage areas, finish types, materials, dimensions, power specs, prices
- Check and report product availability and stock levels from search results
- When products are found with low stock or are unavailable, search for and suggest alternative products
- Compare products from search results and recommend the best options based on customer needs
- Use technical terminology appropriately

When answering:
- ALWAYS use the search tool to find current product information
- Base your responses on actual search results from the product database
- Include specific product names, SKUs, prices, and stock levels from search results
- When stock is low (under 10 units) or unavailable, proactively search for alternatives
- Be professional, precise, and thorough
- Cite specific products found in the search results

Example response structure:
"Based on my search of our product database, I found the following options:
- [Product Name] ([SKU]): [specs] - $[price] - [stock status]
- [Alternative if needed]
Let me know if you need more details on any of these products!"
"""

print("✅ Product Inventory Agent instructions defined!")
print(f"📝 Instruction length: {len(product_inventory_instructions)} characters")

### Define Customer Service Agent Instructions

Now let's define the instructions for our Customer Service Agent. This agent will be friendly, emoji-driven, and focused on customer engagement with a structured response format.

In [ ]:
# Customer Service Agent Instructions
customer_service_instructions = """You are a Customer Service Agent for Zava, a hardware and home improvement company.

Your role:
- Help customers with general inquiries, product selection, and personalized recommendations
- Use the search tool to find products that match customer needs
- Provide warm, friendly, and helpful assistance based on actual product data
- Guide customers to the right products for their projects using search results

CRITICAL - Response Format (ALWAYS follow this structure):
1. Start with a relevant emoji that matches the topic (🔨 for hammers, 🎨 for paint, 🔧 for tools, etc.)
2. Use the search tool to find relevant products
3. Provide a clear, factual response based on search results
4. End with a helpful follow-up question to continue assisting them

Example response structure:
"🔨 [Emoji] I found some great options in our catalog! [Search-based recommendations with specific product names and prices]. What type of project are you working on? [Follow-up question]"

When answering:
- ALWAYS use the search tool to find current product information
- ALWAYS use the emoji + search-based response + follow-up question format
- Use friendly, conversational language
- Focus on helping solve the customer's problem based on actual products
- Ask clarifying questions to better understand their needs
- Be empathetic and customer-focused
- Keep responses concise but informative
- Reference specific products from search results
"""

print("✅ Customer Service Agent instructions defined!")
print(f"📝 Instruction length: {len(customer_service_instructions)} characters")

### Create the Agents

Now we'll create both agents using the instructions we defined above. Notice how much cleaner this code is!

In [ ]:
# Create Product Inventory Agent with Search Access
product_inventory_agent = agent_client.create_agent(
    name="Zava-Inventory-Agent",
    instructions=product_inventory_instructions,  # ⭐ Using the instructions variable defined above
    tools=[search_tool]  # ⭐ Azure AI Search tool
)

print("✅ Product Inventory Agent created!")
print(f"   Agent Name: {product_inventory_agent.name}")
print(f"   Tools Configured: 1 (Azure AI Search via HostedFileSearchTool)")
print(f"   Search Index: {search_index_name}")

# Create Customer Service Agent with Search Access
customer_service_agent = agent_client.create_agent(
    name="Zava-Customer-Agent",
    instructions=customer_service_instructions,  # ⭐ Using the instructions variable defined above
    tools=[search_tool]  # ⭐ Same search tool, different behavior
)

print("\n✅ Customer Service Agent created!")
print(f"   Agent Name: {customer_service_agent.name}")
print(f"   Tools Configured: 1 (Azure AI Search via HostedFileSearchTool)")
print(f"   Search Index: {search_index_name}")

print("\n🎉 Multi-agent team with Azure AI Search is ready!")
print("\n💡 Both agents share the same search tool but use it differently based on their instructions!")

## Step 8: Test Individual Agents with Search

Before orchestrating them together, let's test each agent individually to see how they use Azure AI Search differently.

### Test 1: Product Inventory Agent with Search
Let's ask the Product Inventory Agent to search for technical details and availability.

In [ ]:
# Test Product Inventory Agent with search
question = "I need to paint my bathroom wall. What kinds of paints should I use?"

print(f"❓ Question: {question}\n")
print("📦 Product Inventory Agent Response (with search):")
print("=" * 80)

result = await product_inventory_agent.run(question)
print(result.text)
print("=" * 80)

print("\n💡 Notice how the agent searches the product database and provides specific product details!")

### Test 2: Customer Service Agent with Search
Now let's ask the Customer Service Agent the same question to see the different approach with emoji and follow-up.

In [ ]:
# Test Customer Service Agent with search
print(f"❓ Question: {question}\n")
print("😊 Customer Service Agent Response (with search):")
print("=" * 80)

result = await customer_service_agent.run(question)
print(result.text)
print("=" * 80)

print("\n💡 Notice the different approaches:")
print("   • Product Inventory: Technical search-based details with SKUs and specifications")
print("   • Customer Service: Emoji + friendly search-based recommendations + follow-up question")

## Step 9: Multi-Agent Collaboration with Search

Now let's create scenarios where we route questions to the appropriate agent based on the type of inquiry. Both agents will use Azure AI Search to provide accurate, up-to-date information.

We'll create a simple **router function** that:
1. Analyzes the customer's question
2. Routes it to the most appropriate agent
3. Returns the specialized, search-enhanced response

This demonstrates how you can build intelligent multi-agent systems where each agent has access to the same data source but uses it differently!

In [ ]:
async def route_to_agent(question: str):
    """
    Simple router that determines which agent should handle the question.
    
    Args:
        question: The customer's question
        
    Returns:
        Tuple of (agent_name, response)
    """
    # Keywords for technical/inventory queries
    technical_keywords = [
        "spec", "specification", "dimension", "size", "power", "voltage", 
        "capacity", "stock", "availability", "alternative", "compare",
        "technical", "details", "features", "model", "sku"
    ]
    
    # Keywords for general/service queries
    service_keywords = [
        "help", "recommend", "best", "what should", "which one", "advice",
        "looking for", "need", "want", "project", "use for"
    ]
    
    question_lower = question.lower()
    
    # Count keyword matches
    technical_score = sum(1 for kw in technical_keywords if kw in question_lower)
    service_score = sum(1 for kw in service_keywords if kw in question_lower)
    
    # Route based on scores
    if technical_score > service_score:
        response = await product_inventory_agent.run(question)
        return ("Product Inventory Agent", response.text)
    else:
        response = await customer_service_agent.run(question)
        return ("Customer Service Agent", response.text)

In [ ]:
# Test the router with different types of questions
test_questions = [
    "What are the specifications for your cordless drills?",
    "I'm looking for paint for my bedroom. Can you help?",
    "Do you have any sanders in stock? I need alternatives if out of stock.",
    "What's the best hammer for general home repairs?",
]

print("🔀 Testing Multi-Agent Router with Azure AI Search\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n### Test {i} ###")
    print(f"❓ Question: {question}\n")
    
    agent_name, response = await route_to_agent(question)
    
    print(f"🤖 Routed to: {agent_name}")
    print(f"💬 Response:\n{response}")
    print("=" * 80)

## Step 10: Interactive Multi-Agent Session

Now you can ask your own questions! The router will automatically direct your query to the most appropriate agent, and both agents have access to Azure AI Search for accurate product information.

Try asking about:
- Specific product types (drills, hammers, paint, sanders, etc.)
- Technical specifications
- Product availability
- Recommendations for projects
- Comparisons between products

In [ ]:
# Interactive session
async def interactive_session():
    """
    Run an interactive session where you can ask questions.
    The system will route to the appropriate agent.
    """
    print("🎯 Interactive Multi-Agent Session with Azure AI Search")
    print("Ask questions about Zava's products, or type 'exit' to quit.\n")
    
    # Example questions to get started
    print("💡 Try asking things like:")
    print("   - 'Show me drills under $100'")
    print("   - 'What sanders do you have in stock?'")
    print("   - 'I need paint for a 200 sq ft room'")
    print("   - 'Compare your cordless drills'\n")
    print("=" * 80)
    
    while True:
        # Get user input
        user_question = input("\n❓ Your Question (or 'exit'): ").strip()
        
        if user_question.lower() in ['exit', 'quit', 'q']:
            print("\n👋 Thank you for using Zava's multi-agent system!")
            break
            
        if not user_question:
            print("⚠️  Please enter a question.")
            continue
        
        # Route and get response
        print(f"\n🔍 Searching product database and routing your question...\n")
        agent_name, response = await route_to_agent(user_question)
        
        print(f"🤖 Routed to: {agent_name}")
        print(f"💬 Response:\n{response}")
        print("=" * 80)

# Uncomment the line below to start an interactive session
await interactive_session()

print("✅ Interactive session function is ready!")
print("💡 Uncomment and run 'await interactive_session()' in the cell above to start asking questions!")

## Step 11: Understanding the Search Process

Let's examine how Azure AI Search is being used behind the scenes:

### How the Search Tool Works:
1. **Agent receives question** → Agent analyzes user intent
2. **Agent invokes search tool** → Searches the `zava-products` index
3. **Azure AI Search returns results** → Top 5 relevant products
4. **Agent processes results** → Formats response based on personality
5. **User receives answer** → With actual product data

### Search Configuration:
- **Index**: `zava-products` (product catalog)
- **Query Type**: Simple (straightforward keyword matching)
- **Top K**: 5 results (most relevant products)
- **Automatic**: Agents decide when to search based on the question

### Benefits of This Architecture:
- ✅ **Always current**: Agents access live product data
- ✅ **Accurate**: Responses based on real inventory
- ✅ **Scalable**: Can handle large product catalogs
- ✅ **Flexible**: Each agent interprets search results differently
- ✅ **Specialized**: Right agent for the right question

## Step 12: Advanced Search Examples

Let's test some more complex scenarios that demonstrate the power of Azure AI Search with multi-agent systems.

In [ ]:
# Advanced search scenarios
advanced_questions = [
    {
        "question": "I need to compare all your cordless drills. Which one has the best value?",
        "expected_agent": "Product Inventory Agent",
        "demonstrates": "Complex search with comparison logic"
    },
    {
        "question": "What paint products do you recommend for an outdoor deck project?",
        "expected_agent": "Customer Service Agent", 
        "demonstrates": "Project-based recommendation with search"
    },
    {
        "question": "Show me all measuring tools you have in stock with their prices.",
        "expected_agent": "Product Inventory Agent",
        "demonstrates": "Category search with pricing"
    },
]

print("🎯 Advanced Search Examples\n")
print("=" * 80)

for i, scenario in enumerate(advanced_questions, 1):
    print(f"\n### Advanced Example {i}: {scenario['demonstrates']} ###")
    print(f"❓ Question: {scenario['question']}")
    print(f"🎯 Expected Agent: {scenario['expected_agent']}\n")
    
    agent_name, response = await route_to_agent(scenario['question'])
    
    print(f"🤖 Actual Agent: {agent_name}")
    print(f"💬 Response:\n{response}")
    print("=" * 80)

## Summary and Next Steps

Congratulations! 🎉 You've successfully created a multi-agent system with Azure AI Search using the Microsoft Agent Framework!

### What You Learned:
1. ✅ How to configure `HostedFileSearchTool` for Azure AI Search
2. ✅ How to create specialized agents with search capabilities
3. ✅ How to build a simple routing system for multi-agent collaboration
4. ✅ How different agents can use the same search tool differently
5. ✅ How to integrate Azure AI Search with the Agent Framework

### Key Concepts:
- **Azure AI Search Integration**: Using `HostedFileSearchTool` with `index_name` in `additional_properties`
- **Multi-Agent Architecture**: Specialized agents with shared tools
- **Search-Enhanced Responses**: Real-time data from product catalogs
- **Agent Personality**: Same data, different presentation styles
- **Intelligent Routing**: Directing queries to the right specialist

### Next Steps:
- Try the interactive session to ask your own questions
- Modify agent instructions to change their behavior
- Experiment with different search parameters (top_k, query_type)
- Add more agents with different specializations
- Explore filtering and advanced search queries

### Additional Resources:
- [Microsoft Agent Framework Documentation](https://learn.microsoft.com/en-us/agent-framework/)
- [Azure AI Search Documentation](https://learn.microsoft.com/en-us/azure/search/)
- [Agent Framework GitHub Examples](https://github.com/microsoft/agent-framework/tree/main/python/samples)

---

**Happy building with multi-agent systems!** 🚀